In [7]:
import json
import requests

In [8]:
with open("data/actual/twitter_timeline_385.json","r") as f:
    data=json.load(f)

In [9]:
tweets = data["1217043540208783362"][0]["full_text"]
rc=data["1217043540208783362"][0]["retweet_count"]

In [10]:
# tweets
json_ ={'input_tweet':tweets, "model":"D"}
# json_
print(json.dumps(json_))

{"input_tweet": "RT @Azad_India20: #\u0930\u093e\u092e\u0926\u0947\u0935_\u092c\u093e\u092c\u093e_\u092a\u0948\u0938\u0947_\u0928\u093f\u0915\u093e\u0932\u094b\n\u0930\u093e\u092e\u0926\u0947\u0935 \u092c\u093e\u092c\u093e \u0926\u0947\u0936 \u0915\u0947 10 \u0938\u092c\u0938\u0947 \u0905\u092e\u0940\u0930 \u0932\u094b\u0917\u094b\u0902 \u092e\u0947\u0902 \u0936\u0941\u092e\u093e\u0930 Businesman \u0939\u0948, \u092f\u0947 \u092c\u093e\u0924 \u0905\u0932\u0917 \u0939\u0948 \u0915\u0940 \u0935\u094b \u092a\u0948\u0938\u094b\u0902 \u0915\u0947 \u0932\u093f\u090f\u2026", "model": "D"}


In [14]:
'''
Json format for hate detection api:
[
{'input_tweet': 'tweet text'},
{'input_tweet': 'tweet text'},...
]
Response from api:
[
{'input_tweet': 'tweet text',
'pred_class': 0/1/2,
'pred_label': 'hate speech'/'offensive language'/'neither'},...
]
'''
# json_ = [{'input_tweet':t.text} for t in tweets]
# print(json.dumps(json_))
r = requests.post(url="http://0.0.0.0:8090/api/predict_single", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
# print(r.text)
json.loads(r.text)["pred_class"]
# labels = [t['pred_class']  for t in json.loads(r.text)]
# return labels

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [12]:
hate_nonhate_ratio={}
# retweet_counts={}
count=1
for each_user in data:
    print(count)
    count+=1
    hate_spread_score=0
    non_hate_spread_score = 0
    followers = data2[each_user]["followers"]
    for each_tweet in data[each_user]:
        tweet_data = each_tweet["full_text"]
        retweet_count = each_tweet["retweet_count"]
        retweet_count += 1
        json_ = {'input_tweet':tweet_data, "model":"D"}
        r = requests.post(url="http://0.0.0.0:8090/api/predict_single", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
        label = json.loads(r.text)["pred_class"]
        if label==0 or label==1:
            hate_spread_score+=retweet_count/followers
        else:
            non_hate_spread_score+=retweet_count/followers
    if hate_spread_score and non_hate_spread_score:
        ratio = hate_spread_score/non_hate_spread_score
    else:
        ratio=0
    hate_nonhate_ratio[each_user]=ratio

1


NameError: name 'data2' is not defined

In [241]:
len(hate_nonhate_ratio)

382

In [242]:
len(data)

382

In [247]:
no_data=set(follower_followee_ratio.keys()).difference(set(hate_nonhate_ratio.keys()))
no_data

{'28997944', '853241196910698496', '91254133'}

In [248]:
for uid in no_data:
    hate_nonhate_ratio[uid]=0

In [251]:
with open("data/results/hate_nonhate_ration385.json","w") as f:
    json.dump(hate_nonhate_ratio,f,indent=True)

In [302]:
adoptor_preference_user={}
retweet_norm={}
max_overall=0
count=1
for each_user in data:
    print(count)
    count+=1
    hate_tweets=0
    all_tweets = 0
    max_rt=0
    for each_tweet in data[each_user]:
        tweet_data = each_tweet["full_text"]
        retweet_count = each_tweet["retweet_count"]
        retweet_count += 1
        if retweet_count>max_rt:
            max_rt=retweet_count
        json_ = {'input_tweet':tweet_data, "model":"D"}
        r = requests.post(url="http://0.0.0.0:8090/api/predict_single", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
        label = json.loads(r.text)["pred_class"]
        if label==0 or label==1:
            hate_tweets+=1
        all_tweets+=1
    if all_tweets and hate_tweets:
        ratio = hate_tweets/all_tweets
    else:
        ratio=0
    if max_rt>max_overall:
        max_overall=max_rt
    adoptor_preference_user[each_user]=ratio
    retweet_norm[each_user]=max_rt
for each_user in retweet_norm:
    retweet_norm[each_user]=retweet_norm[each_user]/max_overall

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [303]:
for uid in no_data:
    adoptor_preference_user[uid]=0.0
    retweet_norm[uid]=0.0

In [305]:
len(adoptor_preference_user)

385

In [306]:
len(retweet_norm)

385

In [324]:
# pth.g[432474709]
# pth.g.in_edges(589089799)
profile_user={}
for each_node in adoptor_preference_user:
    in_edges=pth.g.in_edges(int(each_node))
    pp_sum = 0
    count = 0
    for each_edge in in_edges:
#         print(each_edge[0])
        in_node = str(each_edge[0])
        if adoptor_preference_user[in_node] and retweet_norm[in_node]:
            pp_sum += adoptor_preference_user[in_node]/retweet_norm[in_node]
        else:
            pp_sum +=0
        count +=1
    if count:
        profile_user[each_node] = pp_sum/count
    else:
        profile_user[each_node] = 0

In [317]:
with open("data/actual/twitter_following_385.json","r") as f:
            data3=json.load(f)

In [319]:
data3["589089799"]

['409655680']

In [325]:
len(profile_user)

385

In [326]:
with open("data/results/adoptor_user385.json","w") as f:
    json.dump(adoptor_preference_user,f,indent=True)

In [327]:
with open("data/results/profile_user385.json","w") as f:
    json.dump(profile_user,f,indent=True)